In [ ]:
import pandas as pd
from datetime import datetime
import pymongo
from pymongo import MongoClient
from bson.son import SON

In [ ]:
client = MongoClient('localhost', 27017)
db = client.DataManagement
treni_regionali = db.TreniRegionali
altri_treni = db.AltriTreni

Calcolo della currency media per ogni collection.

In [ ]:
pipeline = [{"$unwind": {'path': '$fermate', 'preserveNullAndEmptyArrays': True}},
            {"$group": {"_id": {"id": "$_id", "n_t": "$numero_treno"},
                            "ora": {"$max": {"$cond":[{"$eq":["$fermate.tipo_fermata", "A"]},
                                            "$fermate.arrivo_teorico", 0]
                                            }}}}]

lista_regionali = pd.DataFrame(list(treni_regionali.aggregate(pipeline)))
lista_altri_treni = pd.DataFrame(list(altri_treni.aggregate(pipeline)))

In [ ]:
lista_regionali = lista_regionali[lista_regionali["ora"]!=0]
lista_altri_treni = lista_altri_treni[lista_altri_treni["ora"]!=0]

In [ ]:
for orario in lista_regionali["ora"]:
    datetime.strptime(str(orario), "%H:%M")

for orario in lista_altri_treni["ora"]:
    datetime.strptime(str(orario), "%H:%M")

In [ ]:
lista_regionali["ora_ok"] = lista_regionali["ora"].apply(lambda x: datetime.strptime(str(x), "%H:%M"))
lista_altri_treni["ora_ok"] = lista_altri_treni["ora"].apply(lambda x: datetime.strptime(str(x), "%H:%M"))

In [ ]:
lista_regionali["differenza"] = lista_regionali["ora_ok"].apply(lambda x: (datetime.strptime("22:50","%H:%M") - x)
                                                                                if (datetime.strptime("22:50","%H:%M") - x)>(datetime.strptime("22:50","%H:%M") - datetime.strptime("22:50","%H:%M"))
                                                                                else (datetime.strptime("22:50","%H:%M") - datetime.strptime("22:50","%H:%M")))

lista_altri_treni["differenza"] = lista_altri_treni["ora_ok"].apply(lambda x: (datetime.strptime("22:50","%H:%M") - x)
                                                                                if (datetime.strptime("22:50","%H:%M") - x)>(datetime.strptime("22:50","%H:%M") - datetime.strptime("22:50","%H:%M"))
                                                                                else (datetime.strptime("22:50","%H:%M") - datetime.strptime("22:50","%H:%M")))

In [ ]:
media_regionali = lista_regionali["differenza"].mean()
media_altri = lista_altri_treni["differenza"].mean()